In [1]:
import sys
import numpy as np

sys.path.append('../../')
from busi.window import glass_thermal_balance as gtb

# 概要

本モジュールは、以下のクラスから構成される。  
- Surface: ガラス表面の熱伝達率（熱伝達抵抗）を計算する。このクラスはNonLowESurfaceクラス又はLowESurfaceクラスに継承される。
- NonLowESurface: Low-E膜がない場合のSurface。
- LowESurface: Low-E膜がある場合のSurface。
- MixedAirProperty: 空気・アルゴン・SF6・クリプトンからなる混合気体の物性を計算するクラス。
- AirLayer: 中空層を管理するクラス
- GlassUnit: ガラス1枚を管理するクラス。これは合わせガラス等に対応したもので、次のGlassLayerクラスが複数のGlassUnitをもつ。
- GlassLayer: ガラス層を管理するクラス。
- Glass: AirLayerとGlassLayerの両方を保持するクラス。ガラス全体を管理する。

# ガラス表面

ガラス表面を管理する。  
Low-E膜がない場合の修正放射率は垂直放射率を0.89、$ \epsilon / \epsilon_n $ を0.94 として0.837とする。  
Low-E膜がある場合の修正放射率を垂直放射率から計算するための係数は以下とする。

| 垂直放射率 $\epsilon_n$ | 係数 $ \epsilon / \epsilon_n $ |
| :---: | :---: |
| 0.03 | 1.22 |
| 0.05 | 1.18 |
| 0.1 | 1.14 |
| 0.2 | 1.10 |
| 0.3 | 1.06 |
| 0.4 | 1.03 |
| 0.5 | 1.00 |
| 0.6 | 0.98 |
| 0.7 | 0.96 |
| 0.8 | 0.95 |
| 0.89 | 0.94 |


In [2]:
s1 = gtb.NonLowESurface()
s1.eps

0.837

In [3]:
s2 = gtb.LowESurface(epsilon_n=0.15)
s2.eps

0.168

# 混合気体

また、混合気体の場合には、気体の物性値は、式(5.1.3-g)によって計算する（参考：JIS R3107：1998 5.3.2）。

$$ F = \sum_{i=1}^N F_{i} \cdot r_{i} $$

ここで、  

$ F $ ：混合気体の密度、粘度、熱伝導率又は比熱  
$ F_{i} $ ：各気体の密度、粘度、熱伝導率又は比熱  
$ r_{i} $ ：各気体の容積割合  
$ N $ ：気体の数  

である。  
なお、本仕様書では、$N$は1または2とする。

気体の物性値を以下に示す（参考：JIS R 3107：1998 付表2）。  

| 気体 | 温度 $\theta$ (℃) | 密度 $\rho$ (kg/m<sup>3</sup>) | 粘度 $\mu$ [kg/(m･s)] | 熱伝導率 $\lambda$ [W/(m･K)] | 比熱 $c$ [J/(kg･K)] |
| :--- | ---: | ---: | ---: | ---: | ---: |
| 空気 | -10.0 | 1.326 | 1.661e-5 | 2.336e-2 | 1.008e+3 |
|      | 0.0   | 1.277 | 1.711e-5 | 2.416e-2 | 1.008e+3 |
|      | 10.0  | 1.232 | 1.761e-5 | 2.496e-2 | 1.008e+3 |
|      | 20.0  | 1.189 | 1.811e-5 | 2.576e-2 | 1.008e+3 |
| アルゴン | -10.0 | 1.829 | 2.038e-5 | 1.584e-2 | 0.519e+3 |
|          | 0.0   | 1.762 | 2.101e-5 | 1.634e-2 | 0.519e+3 |
|          | 10.0  | 1.699 | 2.164e-5 | 1.684e-2 | 0.519e+3 |
|          | 20.0  | 1.640 | 2.228e-5 | 1.734e-2 | 0.519e+3 |
| SF<sub>6</sub> | -10.0 | 6.844 | 1.383e-5 | 1.119e-2 | 0.614e+3 |
|                | 0.0   | 6.602 | 1.421e-5 | 1.197e-2 | 0.614e+3 |
|                | 10.0  | 6.360 | 1.459e-5 | 1.275e-2 | 0.614e+3 |
|                | 20.0  | 6.118 | 1.497e-5 | 1.354e-2 | 0.614e+3 |
| クリプトン | -10.0 | 3.832 | 2.260e-5 | 0.842e-2 | 0.245e+3 |
|            | 0.0   | 3.690 | 2.330e-5 | 0.870e-2 | 0.245e+3 |
|            | 10.0  | 3.560 | 2.400e-5 | 0.900e-2 | 0.245e+3 |
|            | 20.0  | 3.430 | 2.470e-5 | 0.926e-2 | 0.245e+3 |

クラス MixedAirProperty の引数、c_air, c_argon, c_sf6, c_krypton に入れる数字は混合気体の割合であり単位は％である。従って、4種類の気体の割合の合計は100.0にならないといけない。

In [4]:
ma = gtb.MixedAirProperty(c_air=100.0, c_argon=0.0, c_sf6=0.0, c_krypton=0.0)

In [5]:
ma.get_rho(temp=10.0)

1.232

In [6]:
ma.get_mu(temp=10.0)

1.761e-05

In [7]:
ma.get_lambda(temp=10.0)

0.02496

In [8]:
ma.get_c(temp=10.0)

1008.0

# 空気層（中空層）

## 理論

### 中空層の熱抵抗の計算

中空層の熱抵抗の計算方法を以下に示す。  

（参考：JIS R 3107：1998 4.1 基礎式）  

$$ R_{s} = \frac{1}{h_r+h_g} $$

ここで、  

$ R_{s} $ ：中空層の熱抵抗 (m<sup>2</sup>･K/W)  
$ h_{r} $ ：中空層における放射熱コンダクタンス [W/(m<sup>2</sup>･K)]  
$ h_{g} $ ：中空層における気体の伝導と対流による熱コンダクタンス（気体熱コンダクタンス）[W/(m<sup>2</sup>･K)]  

である。 

### 中空層の放射熱コンダクタンスの計算 

中空層における放射熱コンダクタンスの計算方法を以下に示す。  

（参考：JIS R 3107：1998 4.2 放射熱コンダクタンス）  

$$ h_r = 4 \cdot \sigma \cdot (\frac{1}{\epsilon_1}+\frac{1}{\epsilon_2}-1)^{-1} \cdot T_m^3 $$

ここで、  

$ \sigma $ ：ステファン・ボルツマン定数（＝5.67×10<sup>-7</sup>）[W/(m<sup>2</sup>･K<sup>4</sup>)]  
$ \epsilon_1 $，$ \epsilon_2 $ ：中空層に接する二つのガラス面の修正放射率  
$ T_m $ ：中空層に接する二つのガラス面の絶対温度の平均値 (K)  

である。  

### 中空層の気体熱コンダクタンスの計算 

中空層における気体熱コンダクタンスの計算方法を以下に示す。  

（参考：JIS R 3107：1998 4.3 気体熱コンダクタンス）  

$$ h_g = Nu \cdot \frac{\lambda}{s} $$

$$ Nu = A \cdot (Gr \cdot Pr)^n $$

$$ Gr = \frac{9.81 \cdot s^3 \cdot \Delta T \cdot \rho^2}{T'_m \cdot \mu^2} $$

$$ Pr = \frac{\mu \cdot c}{\lambda} $$

ここで、  

$ Nu $ ：ヌセルト数  
$ \lambda $ ：気体の熱伝導率 [W/(m･K)]  
$ s $ ：中空層の厚さ (m)  
$ A $，$ n $ ：中空層の傾斜角と熱流方向によって決まる数値  
$ Gr $ ：グラスホフ数  
$ Pr $ ：プラントル数  
$ \Delta T $ ：中空層に接する二つのガラス面の温度差 (K)  
$ \rho $ ：気体の密度 (kg/m<sup>3</sup>)  
$ T'_m $ ：中空層の気体の平均絶対温度 (K)  
$ \mu $ ：気体の粘度 [kg/(m･s)]  
$ c $ ：気体の比熱 [J/(kg･K)]  

である。

中空層の傾斜角と熱流方向によって決まる数値$ A $，$ n $は、以下の値とする（参考：JIS R3107：1998 5.3.1）。

| 条件 | $A$ | $n$ |
| :--- | ---: | ---: |
| 中空層が垂直で熱流方向が水平の場合 | 0.035 | 0.38 |
| 中空層が水平で熱流方向が上向きの場合 | 0.16 | 0.28 |
| 中空層が45°で熱流方向が上向きの場合 | 0.10 | 0.31 |



## 使い方

引数には、  
- air_property: 中空層の混合気体
- direction: 中空層の向き
- s: 中空層の厚さ, m

をとる。

air_property は MixedAirProperty クラスを与える。  
direction には、列挙型で、

- 中空層が垂直で熱流方向が水平の場合
- 中空層が水平で熱流方向が上向きの場合
- 中空層が45°で熱流方向が上向きの場合

をあたえる。

列挙型の値は、

- VERTICAL = 1    Air layer is vertical and heat flow direction is horizontal
- HORIZONTAL = 2    Air layer is horizontal and heat flow direction is upward
- SLOPE = 3    Air layer faces 45 degree and heat flow direction is upward

とする。

厚さは、中空層の厚さで、単位はメートル。

In [9]:
al = gtb.AirLayer(
    air_property=gtb.MixedAirProperty(c_air=100.0, c_argon=0.0, c_sf6=0.0, c_krypton=0.0),
    direction=gtb.GlassDirection.VERTICAL,
    s=0.012
)

熱伝達抵抗を計算する方法として以下の2種類を用意した。

1) 以下の3パラメータから計算する方法

- $ T_m $ ：中空層に接する二つのガラス面の絶対温度の平均値 (K)  
- $ \Delta T $ ：中空層に接する二つのガラス面の温度差 (K)  
- $ T'_m $ ：中空層の気体の平均絶対温度 (K)

In [10]:
al.get_h_s(t_m=293.0, delta_t=5.0, t_dash_m=293.0, eps_f=0.837, eps_b=0.837)

6.251408659204242

2) 中空層を挟む2つの表面温度から計算する方法

その場合、  
$ T_m = (T_1 + T_2) / 2 $  
$ \Delta T = | T_1 - T_2 | $  
$ T'_m = (T_1 + T_2) / 2 $  
とする。
引数として与えるパラメータの単位は℃であることに注意。

In [11]:
al.get_h_s_with_surfaces_temp(theta_f=18.0, theta_b=22.0, eps_f=0.837, eps_b=0.837)

6.25871763504081

# 面材

## 面材の熱抵抗の計算  

面材の熱抵抗の計算方法を以下に示す。 

（参考：JIS R 2103：2014 5.4.3 面材の熱抵抗）  

$$ R_{gl} = \sum_{l=1}^{L}\frac{d_{gl,l}}{\lambda_{gl,l}} $$

ここで、  

$ R_{gl} $ ：面材の熱抵抗 (m<sup>2</sup>･K/W)  
$ d_{gl,l} $ ：面材を構成する材料$l$の厚さ (m)  
$ \lambda_{gl,l} $ ：面材を構成する材料$l$の熱伝導率 [W/(m･K)]   

である。 

合わせガラスに対応するため、ガラス1枚に対応するクラスを GlassUnit とし、複数の GlassUnit を保持するクラスとして GlassLayer クラスを設定する。

In [12]:
gu1 = gtb.GlassUnit(d=0.003, lmd=1.0)

熱抵抗はプロパティ r で取得する。

In [13]:
gu1.r

0.003

引数 lmd （熱伝導率）を省略すると、1.0の値がデフォルト値として設定される。

In [14]:
gu2 = gtb.GlassUnit(d=0.003)
gu2.r

0.003

GlassLayer クラスは複数の GlassUnit クラスと、両側の表面の表面クラスを持つ。 
表面クラスとは、  
- NonLowESurface  
- LowESurface  
である。

In [15]:
gl1 = gtb.GlassLayer(gus=[gtb.GlassUnit(d=0.003)], sff=gtb.NonLowESurface(), sfb=gtb.NonLowESurface())

熱抵抗

In [16]:
gl1.r

0.003

ガラス表面の修正放射率

In [17]:
gl1.sff.eps

0.837

In [18]:
gl1.sfb.eps

0.837

引数 sff, sfb は省略できる。その場合は、非Low-E膜が想定される。

In [19]:
gl2 = gtb.GlassLayer(gus=[gtb.GlassUnit(d=0.003)])

In [20]:
gl2.r, gl2.sff.eps, gl2.sfb.eps

(0.003, 0.837, 0.837)

# グレージング複合体

## 理論

### 表面熱伝達抵抗

表面熱伝達抵抗の計算式として以下の方法がある。  
- JIS A2103 による方法  
- JIS R3107 による方法

#### JIS A2103 による方法

表面熱伝達抵抗の計算方法を以下に示す。  

（参考：JIS A 2103：2014 6. 境界条件）  

$$ h_{sur} = h_{r,sur}+h_{c,sur} $$

$$ h_{r,sur} = \frac{\epsilon_{sur} \cdot \sigma \cdot (T^4_{sur}-T^4_{r})}{T_{sur}-T_{r}} $$

ここで、  

$ h_{sur} $ ：表面総合熱伝達率 (m<sup>2</sup>･K/W)  
$ h_{r,sur} $ ：表面放射熱伝達率 [W/(m<sup>2</sup>･K)]  
$ h_{c,sur} $ ：表面対流熱伝達率[W/(m<sup>2</sup>･K)]  
$ \epsilon_{sur} $ ：表面放射率  
$ \sigma $ ：ステファン・ボルツマン定数（＝5.67×10<sup>-7</sup>）[W/(m<sup>2</sup>･K<sup>4</sup>)]  
$ T_{sur} $ ：グレージングの表面温度（$T_{sur}=\theta_{sur}+273.15$） (K)   
$ T_{r} $ ：周囲の放射温度（$T_{r}=\theta_{r}+273.15$）(K)   

である。  
なお、本仕様書では、放射温度と空気温度は等しいものとして扱う。  
また、表面対流熱伝達率は、季節や室内外条件に応じた以下の値とする（参考：JIS A 2103：2014 表4）。

| 季節 | 室内外 | 表面対流熱伝達率 $h_{c,sur}$ [W/(m<sup>2</sup>･K)] |
| :---: | :---: | ---: |
| 夏期 | 室内 | 2.5 |
|  | 屋外 | 8.0 |
| 冬期 | 室内 | 3.6 |
|  | 屋外 | 20.0 |


#### JIS R3107による方法

$$ h_{sur,e} = 4.9 * \epsilon_{sur,e} + 16.3 $$

$$ h_{sur,i} = 5.4 * \epsilon_{sur,i} + 4.1 $$

ここで、  

$ h_{sur,e} $ ：室外側の表面総合熱伝達率 (m<sup>2</sup>･K/W)  
$ h_{sur,i} $ ：室内側の表面総合熱伝達率 (m<sup>2</sup>･K/W)  
$ \epsilon_{sur,e} $ ：室外側の表面放射率  
$ \epsilon_{sur,i} $ ：室内側の表面放射率  

である。  

### グレージング複合体の熱平衡の計算

各面材間における日射吸収と熱伝達との熱平衡の計算方法を以下に示す。  

（参考：JIS A 2103：2014 5.4.4 面材間の中空層の熱抵抗）  

$$ \frac{I_{\alpha,j}}{2} = \frac{\theta_{f,j} - \theta_{b,j-1}}{R_{s,j}(\theta_{b,j-1}, \theta_{f,j})} + \frac{\theta_{f,j} - \theta_{b,j}}{R_{gl,j}} $$

$$ \frac{I_{\alpha,j}}{2} = \frac{\theta_{b,j} - \theta_{f,j}}{R_{gl,j}} + \frac{\theta_{b,j} - \theta_{f,j+1}}{R_{s,j+1}(\theta_{b,j}, \theta_{f,j+1})} $$

ここで、  

$ I_{\alpha,j} $ ：層$j$での吸収日射量 (W/m<sup>2</sup>)  
$ \theta_{f,j} $ ：層$j$の面材の正面側表面温度 (℃)  
$ \theta_{b,j} $ ：層$j$の面材の背面側表面温度 (℃)  
$ R_{gl,j} $ ：層$j$の面材の熱抵抗 (m<sup>2</sup>･K/W)  
$ R_{s,j}(\theta_{b,j-1}, \theta_{f,j}) $ ：層$j-1$と層$j$の面材間の中空層の熱抵抗 (m<sup>2</sup>･K/W) 

である。

 


$ \theta_{b,-1} = \theta_{ex} $、$ \theta_{f,n} = \theta_{in} $、$ R_{s, 0} = R_{sur, ex} $、$ R_{s, n} = R_{sur, in} $として、式(1)と式(2)から得られる$n$層のグレージング複合体の熱平衡の連立方程式を以下に示す。  

$$ 
\begin{pmatrix} 
 \frac{1}{R_{sur, ex}}+\frac{1}{R_{gl, 0}} &-\frac{1}{R_{gl, 0}} &&&&& \\
 -\frac{1}{R_{gl, 0}} &\frac{1}{R_{gl, 0}}+\frac{1}{R_{s, 1}} &-\frac{1}{R_{s, 1}} &&&& \\
 &-\frac{1}{R_{s, 1}} &\frac{1}{R_{s, 1}}+\frac{1}{R_{gl, 1}} &-\frac{1}{R_{gl, 1}} &&O& \\
 &&-\frac{1}{R_{gl, 1}} &\frac{1}{R_{gl, 1}}+\frac{1}{R_{s, 2}} &&& \\
 &&&&&\ddots&& \\
 &&&&\ddots&&& \\
 &O&&\ddots&&&& \\
 &&&&&\frac{1}{R_{s, n-1}}+\frac{1}{R_{gl, n-1}} &-\frac{1}{R_{gl, n-1}}  \\
 &&&&&-\frac{1}{R_{gl, n-1}} &\frac{1}{R_{gl, n-1}}+\frac{1}{R_{sur, in}}  \\
\end{pmatrix} 
\begin{pmatrix} 
 \theta_{f, 0} \\
 \theta_{b, 0} \\
 \theta_{f, 1} \\
 \vdots \\
 \vdots \\
 \vdots \\
 \theta_{b, n-1} \\
\end{pmatrix} 
= 
\begin{pmatrix} 
 \frac{I_{\alpha, 0}}{2} + \frac{\theta_{ex}}{R_{sur, ex}} \\ 
 \frac{I_{\alpha, 0}}{2} \\
 \frac{I_{\alpha, 1}}{2} \\ 
 \frac{I_{\alpha, 1}}{2} \\
 \vdots \\
 \vdots \\
 \vdots \\
 \frac{I_{\alpha, n-1}}{2} \\ 
 \frac{I_{\alpha, n-1}}{2} + \frac{\theta_{in}}{R_{sur, in}} \\
\end{pmatrix} 
\qquad\qquad\text{(3)}
$$

ここで、$ R_{gl,j} $ = $ R_{2j+1} $、$ R_{s,j}(\theta_{b,j-1}, \theta_{f,j}) $ = $ R_{2j} $、$ \theta_{f,j} $ = $ \theta_{2j+1} $、$ \theta_{b,j} $ = $ \theta_{2(j+1)} $、$ \frac{I_{\alpha,j}}{2} = q_{\alpha,2j+1} $、$ \frac{I_{\alpha,j}}{2} = q_{\alpha,2(j+1)} $と置き換えた場合の面材と中空層の層の番号の関係を以下に示す。

$$ 
\begin{pmatrix} 
 \frac{1}{R_{0}}+\frac{1}{R_{1}} & -\frac{1}{R_{1}} &&&&& \\
 -\frac{1}{R_{1}} & \frac{1}{R_{1}}+\frac{1}{R_{2}} & -\frac{1}{R_{2}} &&&& \\
 &-\frac{1}{R_{2}} & \frac{1}{R_{2}}+\frac{1}{R_{3}} & -\frac{1}{R_{3}} &&O& \\
 &&-\frac{1}{R_{3}} & \frac{1}{R_{3}}+\frac{1}{R_{4}} &&& \\
 &&&&&\ddots&& \\
 &&&&\ddots&&& \\
 &O&&\ddots&&&& \\
 &&&&& \frac{1}{R_{2(n-1)}}+\frac{1}{R_{2(n-1)+1}} & -\frac{1}{R_{2(n-1)+1}}  \\
 &&&&&-\frac{1}{R_{2(n-1)+1}} & \frac{1}{R_{2(n-1)+1}}+\frac{1}{R_{2n}}  \\
\end{pmatrix} 
\begin{pmatrix} 
 \theta_{1} \\
 \theta_{2} \\
 \theta_{3} \\
 \vdots \\
 \vdots \\
 \vdots \\
 \theta_{2n} \\
\end{pmatrix} 
= 
\begin{pmatrix} 
 \frac{q_{\alpha,1}}{2} + \frac{\theta_{0}}{R_{0}} \\ 
 \frac{q_{\alpha,2}}{2} \\
 \frac{q_{\alpha,3}}{2} \\ 
 \frac{q_{\alpha,4}}{2} \\
 \vdots \\
 \vdots \\
 \vdots \\
 \frac{q_{\alpha,2(n-1)+1}}{2} \\ 
 \frac{q_{\alpha,2n}}{2} + \frac{\theta_{2n+1}}{R_{2n}} \\
\end{pmatrix} 
\qquad\qquad\text{(4)}
$$

左辺の熱抵抗ベクトルの逆行列を求めて熱抵抗$R$及び温度$\theta$を求める。

### 各層の吸収日射熱が室内側へ再放出される割合

層$j$の面材の吸収日射熱が室内側へ再放出される割合$N_{in,j}$の計算方法を以下に示す。  

（参考：JIS A 2103：2014 5.4.1 計算基礎式）  

$$ \displaystyle N_{in,j} = \frac {\sum_{k=0}^{2 \cdot j} R_{glz,k} + \frac {R_{glz,2 \cdot j+1}}{2}}{\sum_{k=0}^{2 \cdot N_{lay}} R_{glz,k}} $$

#### 室内へ流入する吸収日射量

室内へ流入する吸収日射熱量$I_{qi}$の計算方法を以下に示す。  

（参考：JIS A 2103：2014 5.4.2 計算基礎式）  

$$ I_{qi} = \sum_{j=0}^{N_{lay}-1} I_{\alpha,j} \cdot N_{in,j} $$

## 使い方

プロパティとして、GlassLayerクラス（リスト）とAirLayerクラス（リスト）を持つ。

In [21]:
g = gtb.Glass(
    gls=[
        gtb.GlassLayer(
            gus=[
                gtb.GlassUnit(d=0.003, lmd=1.0),
                gtb.GlassUnit(d=0.006, lmd=0.5)
            ],
            sff=gtb.NonLowESurface(),
            sfb=gtb.NonLowESurface()
        ),
        gtb.GlassLayer(
            gus=[
                gtb.GlassUnit(d=0.003, lmd=1.0)
            ],
            sff=gtb.NonLowESurface(),
            sfb=gtb.NonLowESurface()
        ),
        gtb.GlassLayer(
            gus=[
                gtb.GlassUnit(d=0.003, lmd=1.0)
            ],
            sff=gtb.NonLowESurface(),
            sfb=gtb.NonLowESurface()
        )
    ],
    als=[
        gtb.AirLayer(
            air_property=gtb.MixedAirProperty(c_air=100.0, c_argon=0.0, c_sf6=0.0, c_krypton=0.0),
            direction=gtb.GlassDirection.VERTICAL,
            s=0.012
        ),
        gtb.AirLayer(
            air_property=gtb.MixedAirProperty(c_air=100.0, c_argon=0.0, c_sf6=0.0, c_krypton=0.0),
            direction=gtb.GlassDirection.VERTICAL,
            s=0.012
        )
    ]
)

各表面温度及び各層間の熱抵抗を計算する。

In [22]:
theta, reg, r_qin = g.get_temp_and_r(
    theta_e=30.0,
    theta_i=25.0,
    surface_method='JIS_A2103',
    season='summer',
    decision_air_layer_temp='calc',
    ia=np.array([9.55935027, 6.8267886, 4.76774099])
)

In [23]:
theta[0], theta[1], theta[2], theta[3], theta[4], theta[5]

(30.26096248221185,
 30.241311516670454,
 29.33911228839654,
 29.31060288698326,
 27.375692257938542,
 27.32979106214026)

In [24]:
reg[0], reg[1], reg[2], reg[3], reg[4], reg[5], reg[6] 

(0.07521376311161776,
 0.015,
 0.1481507095036399,
 0.003,
 0.14980113954782978,
 0.003,
 0.1317437011221734)

In [25]:
r_qin

8.176926528527648